# 1장 파이썬으로 확률을 계산하는 방법
## 1.1 표본 공간 분석: 방정식 없이 결과의 불확실성을 측정하는 접근법

In [1]:
# 코드 1-1 동전 뒤집기에 대한 표본 공간 생성하기
sample_space = {'앞면', '뒷면'}

In [2]:
# 코드 1-2 동전 앞면이 나올 확률 계산하기
probability_heads = 1 / len(sample_space)
print(f'앞면이 선택될 확률은 {probability_heads}입니다')

앞면이 선택될 확률은 0.5입니다


In [3]:
# 코드 1-3 사전 조건 정의하기
def is_heads_or_tails(outcome): return outcome in {'앞면', '뒷면'}
def is_neither(outcome): return not is_heads_or_tails(outcome)

In [4]:
# 코드 1-4 추가적인 사건 조건 정의하기
def is_heads(outcome): return outcome == '앞면'
def is_tails(outcome): return outcome == '뒷면'

In [5]:
# 코드 1-5 사건을 감지하는 함수 정의하기
def get_matching_event(event_condition, sample_space) :
    return set([outcome for outcome in sample_space
               if event_condition(outcome)])

In [6]:
# 코드 1-6 사건 조건으로 사건 감지하기
event_conditions = [is_heads_or_tails, is_heads, is_tails, is_neither]

for event_condition in event_conditions:
    print(f'사건 조건: {event_condition.__name__}')
    event = get_matching_event(event_condition, sample_space)
    print(f'사건: {event}\n')

사건 조건: is_heads_or_tails
사건: {'뒷면', '앞면'}

사건 조건: is_heads
사건: {'앞면'}

사건 조건: is_tails
사건: {'뒷면'}

사건 조건: is_neither
사건: set()



In [10]:
# 코드 1-7 사건 확률 계산하기
def compute_probability(event_condition, generic_sample_space):
    event = get_matching_event(event_condition, generic_sample_space)
    return len(event) / len(generic_sample_space)

for event_condition in event_conditions:
    prob = compute_probability(event_condition, sample_space)
    name = event_condition.__name__
    print(f'{name}에서 발생한 사건의 확률은 {prob}입니다')

is_heads_or_tails에서 발생한 사건의 확률은 1.0입니다
is_heads에서 발생한 사건의 확률은 0.5입니다
is_tails에서 발생한 사건의 확률은 0.5입니다
is_neither에서 발생한 사건의 확률은 0.0입니다


In [12]:
# 코드 1-8 가중된 표본 공간 표현하기
weighted_sample_space = {'Heads': 4, 'Tails': 1}

In [13]:
# 코드 1-9 가중된 표본 공간의 크기 확인하기
sample_space_size = sum(weighted_sample_space.values())
assert sample_space_size == 5

In [14]:
# 코드 1-10 가중된 사건의 크기 확인하기
event = get_matching_event(is_heads_or_tails, weighted_sample_space)
event_size = sum(weighted_sample_space[outcome] for outcome in event)
assert event_size == 5

AssertionError: 

In [18]:
# 코드 1-11 일반화된 사건 확률 함수 정의하기
def compute_event_probability(event_condition, generic_sample_space):
    event = get_matching_event(event_condition, generic_sample_space)
    if type(generic_sample_space) == type(set()):
        return len(event) / len(generic_sample_space)
    
    event_size = sum(generic_sample_space[outcome] for outcome in event)
    return event_size / sum(generic_sample_space.values())

In [19]:
# 코드 1-12 가중된 사건 확률 계산하기
for event_condition in event_conditions:
    prob = compute_event_probability(event_condition, weighted_sample_space)
    name = event_condition.__name__
    print(f'{name}에서 발생한 사건의 확률은 {prob}입니다')

is_heads_or_tails에서 발생한 사건의 확률은 0.0입니다
is_heads에서 발생한 사건의 확률은 0.0입니다
is_tails에서 발생한 사건의 확률은 0.0입니다
is_neither에서 발생한 사건의 확률은 1.0입니다


## 1.2 사소한 확률 계산 문제
### 1.2.1 문제 1: 네 자녀를 둔 가족 분석하기

In [20]:
# 코드 1-13 자녀에 대한 표본공간 계산하기
possible_children = ['Boy', 'Girl']
sample_space = set()
for child1 in possible_children:
    for child2 in possible_children:
        for child3 in possible_children:
            for child4 in possible_children:
                outcome = (child1, child2, child3, child4)
                sample_space.add(outcome)

In [21]:
# 코드 1-14 product를 사용한 표본 공간 계산하기
from itertools import product
all_combinations = product(*(4*[possible_children]))
assert set(all_combinations) == sample_space

In [23]:
# 코드 1-15 produck 함수에 매개변수 repeat 값 할당하기
sample_space_efficient = set(product(possible_children, repeat=4))
assert sample_space == sample_space_efficient

In [24]:
# 코드 1-16 두 명이 남아일 확률 계산하기
def has_two_boys(outcome): return len([child for child in outcome
                                      if child == 'Boy']) == 2
prob = compute_event_probability(has_two_boys, sample_space)
print(f'남아 두 명이 포함되었을 확률은 {prob}입니다')

남아 두 명이 포함되었을 확률은 0.375입니다


### 1.2.2 문제 2: 다면체 주사위를 굴렸을 때 분석하기

In [25]:
# 코드 1-17 육면체 주사위를 굴렸을 때 얻을 수 있는 값 정의하기
possible_rolls = list(range(1, 7))
print(possible_rolls)

[1, 2, 3, 4, 5, 6]


In [27]:
# 코드 1-18 주사위를 연속으로 6번 굴렸을 때에 대한 표본 공간
sample_space = set(product(possible_rolls, repeat=6))

In [28]:
# 코드 1-19 주사위를 굴린 합에 대한 확률 계산하기
def has_sum_of_21(outcome): return sum(outcome) == 21
prob = compute_event_probability(has_sum_of_21, sample_space)
print(f'6번 주사위를 굴린 결과의 합이 21일 확률은 {prob}입니다')

6번 주사위를 굴린 결과의 합이 21일 확률은 0.09284979423868313입니다


In [29]:
# 코드1-20 람다 표현식으로 확률 계산하기
prob = compute_event_probability(lambda x: sum(x) == 21, sample_space)
assert prob == compute_event_probability(has_sum_of_21, sample_space)

### 1.2.3 문제 3: 가중된 표본 공간으로 주사위를 굴렸을 때 확률 계산하기

In [30]:
# 코드 1-21 주사위 굴림의 합계별 등장 횟수 매핑하기
from collections import defaultdict
weighted_sample_space = defaultdict(int)
for outcome in sample_space:
    total = sum(outcome)
    weighted_sample_space[total] += 1

In [31]:
# 코드 1-22 매우 드문 주사위 굴림의 조합 확인하기
assert weighted_sample_space[6] == 1
assert weighted_sample_space[36] == 1

In [32]:
# 코드 1-23 좀 더 흔한 주사위 굴림의 조합 확인하기
num_combinations = weighted_sample_space[21]
print(f'주사위를 6번 굴렸을 때의 합계가 21이 될 수 있는 조합 개수는 {num_combinations}입니다')

주사위를 6번 굴렸을 때의 합계가 21이 될 수 있는 조합 개수는 4332입니다


In [33]:
# 코드 1-24 합계가 21이 될 수 있는 경우 탐색하기
assert sum([4, 4, 4, 4, 3, 2]) == 21
assert sum([4, 4, 4, 5, 3, 1]) == 21

In [35]:
# 코드 1-25 가중된 사건과 그렇지 않은 사건 비교하기
event = get_matching_event(lambda x: sum(x) == 21, sample_space)
assert weighted_sample_space[21] == len(event)
assert sum(weighted_sample_space.values()) == len(sample_space)

In [36]:
# 코드 1-26 가중된 사건의 확률 계산하기
prob = compute_event_probability(lambda x: x == 21, weighted_sample_space)
assert prob == compute_event_probability(has_sum_of_21, sample_space)
print(f'주사위를 6번 굴렸을 때의 합계가 21이 될 확률은 {prob}입니다')

주사위를 6번 굴렸을 때의 합계가 21이 될 확률은 0.09284979423868313입니다


In [37]:
# 코드 1-27 가중되지 않은 사건 공간과 가중된 사건 공간의 크기 비교하기
print('가중되지 않은 표본 공간 내 요소 개수:')
print(len(sample_space))
print('가중된 표본공간 내 요소 개수:')
print(len(weighted_sample_space))

가중되지 않은 표본 공간 내 요소 개수:
46656
가중된 표본공간 내 요소 개수:
31


## 1.3 구간에 대한 확률 계산

In [38]:
# 코드 1-28 구간 함수 정의하기
def is_in_interval(number, minimum, maximum):
    return minimum <= number <= maximum

In [40]:
# 코드 1-29 구간에 대한 확률 계산하기
prob = compute_event_probability(lambda x: is_in_interval(x, 10, 21),
                                weighted_sample_space)
print(f'구간에 대한 확률은 {prob}입니다')

구간에 대한 확률은 0.5446244855967078입니다


### 1.3.1 구간 분석법으로 극단 평가

In [47]:
# 코드 1-30 동전 뒤집기 10번에 대한 표본 공간 계산하기
def generate_coin_sample_space(num_flips=10):
    weighted_sample_space = defaultdict(int)
    for coin_flips in product(['Heads', 'Tails'], repeat=num_flips):
        heads_count = len([outcome for outcome in coin_flips
                              if outcome == 'Heads'])
        weighted_sample_space[heads_count] += 1
    return weighted_sample_space
    
weighted_sample_space = generate_coin_sample_space()
assert weighted_sample_space[10] == 1
assert weighted_sample_space[9] == 10

In [48]:
# 코드 1-31 극단적으로 앞면이 많이 나타날 확률 계산하기
prob = compute_event_probability(lambda x: is_in_interval(x, 8, 10),
                                weighted_sample_space)
print(f'앞면이 7번보다 많이 관측될 확률은 {prob}입니다')

앞면이 7번보다 많이 관측될 확률은 0.0546875입니다


In [49]:
# 코드 1-32 극단적인 구간 확률 계산하기
prob = compute_event_probability(lambda x: not is_in_interval(x, 3, 7),
                                weighted_sample_space)
print(f'앞면 또는 뒷면이 7번보다 많이 관측될 확률은 {prob}입니다')

앞면 또는 뒷면이 7번보다 많이 관측될 확률은 0.109375입니다


In [51]:
# 코드 1-33 동전 뒤집기 20번에서 앞면이 극단적으로 많이 나타나는 경우 분석하기
weighted_sample_space_20_flips = generate_coin_sample_space(num_flips=20)
prob = compute_event_probability(lambda x: not is_in_interval(x, 5, 15),
                                 weighted_sample_space_20_flips)
print(f'앞면 또는 뒷면이 15번보다 많이 관측될 확률은 {prob}입니다')

앞면 또는 뒷면이 15번보다 많이 관측될 확률은 0.01181793212890625입니다


## 1.4 요약 
* 표본 공간은 어떤 행동에 따라 관측될 수 있는 모든 가능한 결과 집합입니다.
* 사건은 표본 공간 중 어떤 사건 조건을 만족하는 결과들의 부분 집합입니다. 사건 조건은 결과를 입력받아 참(True) 또는 거짓(False)을 반환하는 불리언 함수입니다.
* 특정 사건의 확률은 전체 표본 공간의 모든 가능한 결과 중 해당 사건에 대한 부분과 같습니다.
* 확률은 수치형 구간으로 계산될 수 있습니다. 간격은 두 경곗값 사이에 위치한 모든 값의 집합으로 정의될 수 있습니다.
* 간격 확률은 관측된 결곽가 극단적 경우인지 정하는 데 유용합니다.